In [ ]:
from flask import Flask, render_template, Response
from datetime import datetime
import cv2
import numpy as np
import time
import sys
import boto3
import json

service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
region_name = 'kr-standard'
access_key = 'fMwo4ZlCS60SGzwjoxb1'
secret_key = 'kb0CgwTfxQi6uhvdfaT1yxgysOsCZnMbD6xTdC8G'

video_list = []

if __name__ == "__main__":
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)

    bucket_name = 'static-site-test-bucket'

    # list all in the bucket
    max_keys = 300
    response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys)

    while True:
        for content in response.get('Contents'):
            if content.get('Key').startswith("video/") and content.get('Key').endswith(".mp4"):
                print(' Name=%s' % (content.get('Key')))
                video_list.append(content.get('Key'))
        if response.get('IsTruncated'):
            response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys,
                                       Marker=response.get('NextMarker'))
        else:
            break
            

face_cascade = cv2.CascadeClassifier("xml/haarcascade_frontalface_default.xml")

now = datetime.now()
time_string = now.strftime("%Y - %m - %d  %H : %M")

lastTime = time.time()*1000.0
delt = time.time()*1000.0-lastTime
s = str(int(delt))



# 얼굴 탐지 모델 가중치
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascade_default_path = "xml/haarcascade_frontalface_default.xml"
cascade_alt = 'xml/haarcascade_frontalface_alt.xml'
faceCascade = cv2.CascadeClassifier(cascade_default_path);
font = cv2.FONT_HERSHEY_SIMPLEX
age_net = cv2.dnn.readNetFromCaffe(
                                    'weight/deploy_age.prototxt',
                                    'weight/age_net.caffemodel')

gender_net = cv2.dnn.readNetFromCaffe(
                                    'weight/deploy_gender.prototxt',
                                    'weight/gender_net.caffemodel')


# 모델 불러오기
cascade = cv2.CascadeClassifier(cascade_alt)


#인덱스 몇부터 시작할지
id = 0

# names related to ids: example ==> None: id=0,  etc
# 이런식으로 사용자의 이름을 사용자 수만큼 추가해준다.
names = ['None', 'OIJ', 'SYS', 'JSJ', 'JYS', 'ORK']

age_list = ['(0 ~ 2)','(4 ~ 6)','(8 ~ 12)','(15 ~ 20)',
            '(25 ~ 32)','(38 ~ 43)','(48 ~ 53)','(60 ~ 100)']
gender_list = ['Male', 'Female']

#default values
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

video = cv2.VideoCapture(video_list[5])


app = Flask(__name__)
@app.route("/")
def index():
    template_data = {
        'title':'face_recog_flask',
        'time': time_string
        }
    return render_template('index.html', **template_data, video_list = video_list, ment = ment)

def videoDetector(video,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list):

    while True:

        # 캡처 이미지 불러오기
        ret,img = video.read()
        #img = cv2.flip(img, -1) # 카메라 반전
        # 그레이 스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        # 영상 압축
        try:
            img = cv2.resize(img,dsize=None,fx=1.0,fy=1.0)
        except:
            break

        # cascade 얼굴 탐지 알고리즘 
        results = cascade.detectMultiScale(gray,            # 입력 이미지
                                           scaleFactor= 1.1,# 이미지 피라미드 스케일 factor
                                           minNeighbors=5,  # 인접 객체 최소 거리 픽셀
                                           minSize=(20,20)  # 탐지 객체 최소 크기
                                           )

        
        for(x,y,w,h) in results:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

            # 확률 계산 ==> "0" is perfect match
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1) 

        
        
        for box in results:
            x, y, w, h = box
            face = img[int(y):int(y+h),int(x):int(x+h)].copy()
            blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            # gender detection
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = gender_preds.argmax()
            
            # Predict age
            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = age_preds.argmax()
            
            info = gender_list[gender] +' '+ age_list[age]

            cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,255), thickness=2)
            cv2.putText(img,info,(x,y-15),0, 0.5, (0, 255, 0), 1)


         # 영상 출력
        cv2.imshow('facenet',img)
        


        if cv2.waitKey(1) > 0: 
            cv2.destroyAllWindows()
            break

        ret, buffer = cv2.imencode('.jpg', img)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
 

@app.route("/live_feed")
def live_feed():
    age_list = ['(0 ~ 2)','(4 ~ 6)','(8 ~ 12)','(15 ~ 20)',
            '(25 ~ 32)','(38 ~ 43)','(48 ~ 53)','(60 ~ 100)']
    gender_list = ['Male', 'Female']

    #default values
    MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

    cascade = cv2.CascadeClassifier(cascade_alt)
    
    # 영상 파일 
    video = video_list[0]
    age_net = cv2.dnn.readNetFromCaffe(
                                        'weight/deploy_age.prototxt',
                                        'weight/age_net.caffemodel')

    gender_net = cv2.dnn.readNetFromCaffe(
                                        'weight/deploy_gender.prototxt',
                                        'weight/gender_net.caffemodel')

    return Response(videoDetector(video,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list)
                    , mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    app.run(host = "0.0.0.0")